In [37]:
import configparser
import glob
import os
import pandas as pd
import subprocess
import sys
import tarfile 
from urllib.request import urlretrieve
import json
import tempfile
import tensorflow as tf

In [38]:
sys.path.append("../src")
from utils import str_to_value

In [39]:
sys.path.append("../bert")
import modeling

In [40]:
CURDIR = os.getcwd()
CONFIGPATH = os.path.join(CURDIR, os.pardir, 'config.ini')
config = configparser.ConfigParser()
config.read(CONFIGPATH)

['/home/john/hdd/xrosriver/bert-japanese_new/notebook/../config.ini']

In [41]:
FILEURL = config['FINETUNING-DATA']['FILEURL']
FILEPATH = config['FINETUNING-DATA']['FILEPATH']
EXTRACTDIR = config['FINETUNING-DATA']['TEXTDIR']

In [42]:
PRETRAINED_MODEL_PATH = '../model/bert-wiki-ja/model.ckpt-1400000'

In [43]:
bert_config_file = tempfile.NamedTemporaryFile(mode='w+t', encoding='utf-8', suffix='.json')
bert_config_file.write(json.dumps({k:str_to_value(v) for k,v in config['BERT-CONFIG'].items()}))
bert_config_file.seek(0)
bert_config_file_path = str(bert_config_file.name)
bert_config = modeling.BertConfig.from_json_file(bert_config_file.name)

In [44]:
!echo 'すべての人間は、生れながらにして自由であり、かつ、尊厳と権利とについて平等である。 ||| 人間は、理性と良心とを授けられており、互いに同胞の精神をもって行動しなければならない。' > /tmp/input.txt

In [45]:
!python ../src/extract_features.py \
  --input_file=/tmp/input.txt \
  --output_file=output.json \
  --vocab_file=../model/bert-wiki-ja/wiki-ja.vocab \
  --model_file=../model/bert-wiki-ja/wiki-ja.model \
  --bert_config_file={bert_config_file.name} \
  --init_checkpoint={PRETRAINED_MODEL_PATH} \
  --layers=-1,-2,-3,-4 \
  --max_seq_length=128 \
  --batch_size=8

Loaded a trained SentencePiece model.

For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.dense instead.
2019-07-29 12:30:11.630869: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: SSE4.1 SSE4.2 AVX AVX2 FMA
2019-07-29 12:30:11.667507: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 3000000000 Hz
2019-07-29 12:30:11.668354: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x55ac40526d50 executing computations on platform Host. Devices:
2019-07-29 12:30:11.668387: I tensorflow/compiler/xla/service/service.cc:158]   StreamExecutor device (0): <undefined>, <undefined>
2

Let's have a look at the features of the last layer for the word "人間".

The 0-th token is always [CLS], and the 1st token of a sentence is [▁]. So the word comes in 3rd position.

In [59]:
import json

outputs = json.load(open('output.json'))
print(outputs['features'][3]['token'])

人間


The last year is layer 0, the one before is layer -1, etc...
The embeddings are stored in the *values* entry.

In [66]:
embeddings = outputs['features'][3]['layers'][0]['values']
print(len(embeddings))

768
